In [ ]:
%tensorboard --logdir ./logs/fit

In [53]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# # 获取上证50成分股
# rs = bs.query_sz50_stocks(date='2010-04-28')

# # 登出系统
# bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success


In [54]:
df = bs.query_stock_basic().get_data()
dd = bs.query_stock_industry().get_data()
if not df.empty and not dd.empty:
    dataframe = df.merge(dd[["code", "industry"]], on=["code"], how='left')
    dataframe = dataframe.rename(
        columns={
            "code": "code",
            "code_name": "code_name",
            "industry": "industry",
            "ipoDate": "in_date",
            "outDate": "out_date",
            "type": "type",
            "status": "status",
        }
    )
    dataframe["industry"] = dataframe["industry"].replace("", "其他")
    dataframe["type"] = dataframe["type"].map(lambda x: {"1": "股票", "2": "指数", "3": "其它", "4": "可转债", "5": "ETF"}.get(x, "其他"))
    dataframe["status"] = dataframe["status"].map(lambda x: {"1": "上市", "2": "退市"}.get(x, "其他"))

In [61]:
dataframe['code'].map(lambda x: x.split('.')[0]).value_counts()

sz    3881
sh    3068
of     971
Name: code, dtype: int64

In [ ]:
dataframe[dataframe['code'].str.startswith(('sh', 'sz'))]['code'].shape

In [8]:
code = 'sh.600356'

df = bs.query_history_k_data_plus(
                code=code,
                fields="code,date,open,high,low,close,volume,amount,turn",
                start_date='2000-01-01',
                end_date='2024-05-14',
                frequency="d",
                adjustflag="1",
            ).get_data()

In [9]:
df.head()

""


In [12]:
df = bs.query_stock_basic().get_data()
dd = bs.query_stock_industry().get_data()
if not df.empty and not dd.empty:
    dataframe = df.merge(dd[['code', 'code_name', 'industry']], on=['code', 'code_name'])

In [21]:
dataframe['industry'].replace("", "其他").value_counts()

其他      889
机械设备    472
化工      425
医药生物    397
电子      342
计算机     265
电气设备    242
汽车      213
公用事业    205
传媒      166
轻工制造    150
建筑装饰    148
有色金属    134
交通运输    126
食品饮料    120
房地产     115
通信      109
纺织服装     94
农林牧渔     94
商业贸易     92
国防军工     89
非银金融     82
建筑材料     73
家用电器     71
采掘       63
银行       41
综合       38
休闲服务     36
钢铁       36
Name: industry, dtype: int64

In [3]:
dd = bs.query_stock_basic().get_data()

In [7]:
dd['code'].map(lambda x: x.split('.')[0]).value_counts()

sz    3881
sh    3068
of     971
Name: code, dtype: int64

In [2]:
df = bs.query_trade_dates(start_date='2013-02-05', end_date='2014-02-05').get_data()
df = df[df["is_trading_day"] == "1"]["calendar_date"]  # 选择交易日(%Y-%m-%d)

In [6]:
pd.DataFrame([{'datetime': '2013-02-05'}])

,datetime
0,2013-02-05


In [15]:
df = bs.query_all_stock(day='2013-02-05').get_data()

In [27]:
dd = bs.query_trade_dates(start_date='2013-02-05', end_date='2014-02-05').get_data().to_records(index=False)

In [24]:
sample = df.head().to_records(index=False)

In [13]:
dd = bs.query_all_stock()

In [16]:
import sqlite3

db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')

In [18]:
pd.read_sql_query('select * from hh_quant_history_trade_date_info;', db_conn)["datetime"].max()

'2024-05-07'

In [20]:
from datetime import datetime
datetime.strptime('2024-05-07', "%Y-%m-%d")

datetime.datetime(2024, 5, 7, 0, 0)

In [21]:
import tensorflow as tf
import numpy as np

2024-05-07 07:32:43.068797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 07:32:43.966745: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
sample_1 = np.array([1.,2.,3.,0,0,0])
sample_2 = np.array([1.,2.,3.,-1e9,-1e9,-1e9])

print(f"version_1: {tf.nn.softmax(sample_1)}")
print(f"version_2: {tf.nn.softmax(sample_2)}")

version_1: [0.08189353 0.22260971 0.60511592 0.03012695 0.03012695 0.03012695]
version_2: [0.09003057 0.24472847 0.66524096 0.         0.         0.        ]


In [7]:
test = np.array([1,2,3,-65504.])

tf.nn.softmax(test)

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.09003057, 0.24472847, 0.66524096, 0.        ])>

In [8]:
pd.DataFrame({"datetime": [1,2,3,4,5]})

,datetime
0,1
1,2
2,3
3,4
4,5


In [23]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)

Python 3.8.10


In [32]:
db_downloader._download_history_date_factor_info(start_date='2020-01-01', end_date='2021-01-01').head()

,datetime,weekday,day_of_week,day_of_month,month,season
0,2020-01-02,3,Thursday,2,1,Winter
1,2020-01-03,4,Friday,3,1,Winter
2,2020-01-06,0,Monday,6,1,Winter
3,2020-01-07,1,Tuesday,7,1,Winter
4,2020-01-08,2,Wednesday,8,1,Winter


In [6]:
df.head()

,index_code,datetime,open,high,low,close,volume,amount
0,000016,2004-01-02,996.996,1021.568,993.892,1011.347,8.064652e+08,5.742759e+09
1,000016,2004-01-05,1008.279,1060.898,1008.279,1060.801,1.446818e+09,1.070646e+10
2,000016,2004-01-06,1059.141,1086.694,1059.095,1075.656,1.699133e+09,1.241700e+10
3,000016,2004-01-07,1075.562,1095.841,1070.980,1086.303,1.372942e+09,1.035253e+10
4,000016,2004-01-08,1087.680,1108.291,1082.509,1102.662,1.078043e+09,8.188927e+09
